In [3]:
import cv2
import numpy as np

In [4]:
def delete_small_contours(contours):
    new_contours = []
    for contour in contours:
        if(cv2.contourArea(contour) > 10):
            new_contours.append(contour)
    return new_contours

def resize_image(img, size=(28,28)):

    h, w = img.shape[:2]
    c = img.shape[2] if len(img.shape)>2 else 1

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

In [49]:
import cv2 
import numpy as np 
image2 = cv2.imread('amborella081.jpg')
image2 = resize_image(image2, size=(int(image2.shape[0]/4),int(image2.shape[1]/4)))
print(image2.shape)
img_gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

cv2.imshow('LIST',img_gray2)
cv2.waitKey(0)
cv2.destroyAllWindows()
unique, counts = np.unique(img_gray2, return_counts=True)
#print(np.asarray((unique, counts)).T)

ret, thresh2 = cv2.threshold(img_gray2, 200, 255, cv2.THRESH_BINARY)

contours4, hierarchy4 = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
image_copy5 = img_gray2.copy()

color = int(unique[np.argmax(counts)])
cv2.drawContours(image_copy5, delete_small_contours(contours4), -1, (255,255,255), 2, cv2.LINE_AA)
# see the results
cv2.imshow('EXTERNAL', image_copy5)
#print(f"EXTERNAL: {hierarchy4}")
cv2.waitKey(0)
cv2.imwrite('contours_retr_external.jpg', image_copy5)
cv2.destroyAllWindows()


img = cv2.imread('amborella081.jpg')
Z = img.reshape((-1,3))
# convert to np.float32
Z = np.float32(Z)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 5

attemps = 10
ret,label,centers=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
# Now convert back into uint8, and make original image
centers = np.uint8(centers)
res = centers[label.flatten()]
res2 = res.reshape((img.shape))

image_copy6 = image2.copy()
unique, counts = np.unique(label, return_counts=True)

print(np.asarray((unique, counts)).T)
color = (int(centers[np.argmax(counts)][0]),int(centers[np.argmax(counts)][1]),int(centers[np.argmax(counts)][2]))
print(color)
cv2.drawContours(image_copy6, delete_small_contours(contours4), -1, color, -1, cv2.LINE_AA)
# see the results
cv2.imshow('EXTERNAL', image_copy6)
#print(f"EXTERNAL: {hierarchy4}")
cv2.waitKey(0)
cv2.imwrite('contours_retr_external.jpg', image_copy5)
cv2.destroyAllWindows()
img = image_copy6
Z = img.reshape((-1,3))
# convert to np.float32
Z = np.float32(Z)
# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 2

attemps = 10
ret,label,centers=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
# Now convert back into uint8, and make original image
centers = np.uint8(centers)
res = centers[label.flatten()]
res2 = res.reshape((img.shape))
cv2.imshow('contours_retr_external.jpg', res2)
cv2.waitKey(0)
cv2.destroyAllWindows()

img = image2.copy()


for contour in delete_small_contours(contours4):
    approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
    cv2.drawContours(img, [approx], 0, color, 5)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
    if len(approx) == 3:
        cv2.putText( img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0) )
    elif len(approx) == 4 :
        x, y , w, h = cv2.boundingRect(approx)
        aspectRatio = float(w)/h
        print(aspectRatio)
        if aspectRatio >= 0.95 and aspectRatio < 1.05:
            cv2.putText(img, "square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))

        else:
            cv2.drawContours(img,[contour], -1, color, -1, cv2.LINE_AA)

    elif len(approx) == 5 :
        cv2.putText(img, "pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
    elif len(approx) == 10 :
        cv2.putText(img, "star", (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
    else:
        cv2.drawContours(img,[contour], -1, color, -1, cv2.LINE_AA)


cv2.imwrite('fixed/fixed image.jpg', img)







(820, 1293, 3)
[[      0 6722551]
 [      1 2186772]
 [      2 1634470]
 [      3 5047540]
 [      4 1393017]]
(130, 192, 214)
2.6610169491525424
1.5737704918032787


: 

2.6610169491525424
1.5737704918032787
